#Stock Price Prediction using LSTM in PyTorch
Stock price information is a type of time-series data. Both neural and non-neural approaches are helpful for analyzing that kind of data. The approaches for modeling the time-series data are also known as **Sequence Modeling** techniques.

In this code, we use Long Short Term Memeory, known as **LSTM**, for modeling the stock price. Thus, we can predict the stock price for the future using the recurrent neural network.

**Note:** We would like to run this code on GPU. If you are using colab, check out the **Change Runtime Type** option in the **Runtime** menu, right above this text, to make sure that you are using a GPU on Google Colab. 

#YOUR_JOB

Please complete the code denoted by **#YOUR_JOB** according to the comments.

In [ ]:
import numpy as np                          
import pandas as pd   
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from time import time

# 1. Data
We import the dataset using pandas package.We use Amazon Stock Price dataset, named `AMZN_2006-01-01_to_2018-01-01.csv` from [**kaggle DJIA 30 Stock Time Series**](https://www.kaggle.com/szrlee/stock-time-series-20050101-to-20171231). Don't forget to upload it to the colab machine first. 

In [ ]:
print("Getting data ...")

data = pd.read_csv("AMZN_2006-01-01_to_2018-01-01.csv")
print("Data is ready.")

data.head()

#### 1.1. Plots
The raw dataset contains `Date`, `Open`, `High`, `Low`, `Close`, `Volume`, and `Name`. We use the close value for the modeling task. We plot the closing price and its autocorrelation here.

In [ ]:
sns.set_style("darkgrid")
plt.figure(figsize = (15,9))
plt.plot(data[['Close']])
plt.xticks(range(0,data.shape[0],500),data['Date'].loc[::500],rotation=45)
plt.title("Amazon Stock Price",fontsize=18, fontweight='bold')
plt.xlabel('Date',fontsize=18)
plt.ylabel('Close Price (USD)',fontsize=18)
plt.show()

#### 1.2 Preprocessing

In [ ]:
price = data[['Close']]
price.info()

**#YOUR_JOB**

We should rescale the stock price to [-1, 1]. Use `MinMaxScaler` for the conversion.

In [ ]:
scaler = #------ YOUR CODE HERE -----------
price['Close'] = scaler.fit_transform(price['Close'].values.reshape(-1,1))

#### 1.3. Split Data

We should split data into train and test sets. Moreover, remember that the input sequence of length `L` should be converted to a batch of overlapped sequences with length `T` as the input of the model and the same batch size with the length 1 for the output `y`.

**#YOUR JOB**
In the following cell, we make the train set of the first `train_set_size` samples. The remaining samples would be the test set. Please complete the code for the test set. 

In [ ]:
def split_data(stock, lookback):
    data_raw = stock.to_numpy() # convert to numpy array; [L x 1]
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - lookback): 
        data.append(data_raw[index: index + lookback])
    
    data = np.array(data)                               # [N x lookback x 1]
    test_set_size = int(np.round(0.2 * data.shape[0]))
    train_set_size = data.shape[0] - test_set_size
    
    x_train = data[:train_set_size, :-1, :]    # [N_train x T x 1]
    y_train = data[:train_set_size, -1, :]     # [N_train x 1 x 1]
    
    # YOUR_JOB
    # make x_test & y_test with the data array,  
    # from the index of train_set_size to the end. 
    x_test = # -------- YOUR CODE HERE ---------
    y_test = # -------- YOUR CODE HERE ---------
    
    return [x_train, y_train, x_test, y_test]

In [ ]:
lookback = 20 # choose sequence length
x_train, y_train, x_test, y_test = split_data(price, lookback)
print('x_train.shape = ',x_train.shape)
print('y_train.shape = ',y_train.shape)
print('x_test.shape = ',x_test.shape)
print('y_test.shape = ',y_test.shape)

# 2. Model

#### 2.1. Hyper-parameters

In [ ]:
input_dim = 1
hidden_dim = 32
num_layers = 2
output_dim = 1
num_epochs = 100

# specify GPU
device = torch.device("cuda")

#### 2.2. LSTM Class

We define a class, named `StockPredictor`, for the stock price prediction using LSTM. Since the dimension of the LSTM hidden state is `hidden_dim`, we need to convert it to 1-D data as the prediction output. We overwrite the `__init__`
 and `forward` procedures as usual.

**#YOUR_JOB**

The dimensionality convertion from `hidden_dim` to 1-D to make the output prediction can be handled by a linear layer defined as `self.fc` in the following code. Please complete the `forward` function such that the `out` would be to convert the final output with this dimensions: `[N x 1 x 1]`.    

In [ ]:
class StockPredictor(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(StockPredictor, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.lstm(x)
                                                       # in: [N x T x 1]
                                                       # out: [N x T x D*n-dir]
        
        
        out = # ----- YOUR CODE HERE ----------- 
        return out

In [ ]:
model = StockPredictor(input_dim, hidden_dim, num_layers, output_dim)

# push the model to GPU
model = model.to(device)

criterion = torch.nn.MSELoss(reduction='mean')
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

#### 2.3. Train

In [ ]:
x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_train = torch.from_numpy(y_train).type(torch.Tensor)
y_test = torch.from_numpy(y_test).type(torch.Tensor)

In [ ]:
hist = np.zeros(num_epochs)
start_time = time()

for t in range(num_epochs):
    y_train_pred = model(x_train.to(device))

    loss = criterion(y_train_pred, y_train.to(device))
    print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()

    optimiser.zero_grad()
    loss.backward()
    optimiser.step()
    
training_time = time() - start_time
print("Training time: {} min".format(training_time / 60))

#### 2.4. Plots

In [ ]:
predict = pd.DataFrame(scaler.inverse_transform(y_train_pred.cpu().detach().numpy()))
original = pd.DataFrame(scaler.inverse_transform(y_train.numpy()))

In [ ]:
sns.set_style("darkgrid")    

fig = plt.figure()
fig.subplots_adjust(hspace=0.2, wspace=0.2)

plt.subplot(1, 2, 1)
ax = sns.lineplot(x = original.index, y = original[0], label="Data", color='royalblue')
ax = sns.lineplot(x = predict.index, y = predict[0], label="Training Prediction (LSTM)", color='tomato')
ax.set_title('Stock price', size = 14, fontweight='bold')
ax.set_xlabel("Days", size = 14)
ax.set_ylabel("Cost (USD)", size = 14)
ax.set_xticklabels('', size=10)


plt.subplot(1, 2, 2)
ax = sns.lineplot(data=hist, color='royalblue')
ax.set_xlabel("Epoch", size = 14)
ax.set_ylabel("Loss", size = 14)
ax.set_title("Training Loss", size = 14, fontweight='bold')
fig.set_figheight(6)
fig.set_figwidth(16)

# 3. Evaluation

In [ ]:
import math, time
from sklearn.metrics import mean_squared_error

# make predictions
y_test_pred = model(x_test.to(device))

# invert predictions
y_train_pred = scaler.inverse_transform(y_train_pred.cpu().detach().numpy())
y_train = scaler.inverse_transform(y_train.numpy())
y_test_pred = scaler.inverse_transform(y_test_pred.cpu().detach().numpy())
y_test = scaler.inverse_transform(y_test.numpy())

# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(y_train[:,0], y_train_pred[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(y_test[:,0], y_test_pred[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

Train Score: 9.62 RMSE
Test Score: 227.11 RMSE


In [ ]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(price)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[lookback:len(y_train_pred)+lookback, :] = y_train_pred

# shift test predictions for plotting
testPredictPlot = np.empty_like(price)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(y_train_pred)+lookback-1:len(price)-1, :] = y_test_pred

original = scaler.inverse_transform(price['Close'].values.reshape(-1,1))

predictions = np.append(trainPredictPlot, testPredictPlot, axis=1)
predictions = np.append(predictions, original, axis=1)
result = pd.DataFrame(predictions)
result.head()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(go.Scatter(x=result.index, y=result[0],
                    mode='lines',
                    name='Train prediction')))
fig.add_trace(go.Scatter(x=result.index, y=result[1],
                    mode='lines',
                    name='Test prediction'))
fig.add_trace(go.Scatter(go.Scatter(x=result.index, y=result[2],
                    mode='lines',
                    name='Actual Value')))
fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=True,
        showticklabels=False,
        linecolor='white',
        linewidth=2
    ),
    yaxis=dict(
        title_text='Close (USD)',
        titlefont=dict(
            family='Rockwell',
            size=12,
            color='white',
        ),
        showline=True,
        showgrid=True,
        showticklabels=True,
        linecolor='white',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Rockwell',
            size=12,
            color='white',
        ),
    ),
    showlegend=True,
    template = 'plotly_dark'

)



annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Results (LSTM)',
                              font=dict(family='Rockwell',
                                        size=26,
                                        color='white'),
                              showarrow=False))
fig.update_layout(annotations=annotations)

fig.show()